In [1]:
# import the necessary packages
from tensorflow.keras.models import load_model
from collections import deque
import numpy as np
import argparse
import pickle
import cv2

In [2]:
size_dq = 128

# load the trained model and label binarizer from disk
print("[INFO] loading model and label binarizer...")
model = load_model("sports_model.h5")
lb = pickle.loads(open("label_bin.pickle", "rb").read())
# initialize the image mean for mean subtraction along with the
# predictions queue
mean = np.array([123.68, 116.779, 103.939][::1], dtype="float32")
Q = deque(maxlen=size_dq)

[INFO] loading model and label binarizer...


In [4]:
# initialize the video stream, pointer to output video file, and
# frame dimensions
vs = cv2.VideoCapture("input_vid_3.mp4")

writer = None

(W, H) = (None, None)

# loop over frames from the video file stream
while True:
        # read the next frame from the file
        (grabbed, frame) = vs.read()
        
        # if the frame was not grabbed, then we have reached the end
        # of the stream
        if not grabbed:
            break
            
        # if the frame dimensions are empty, grab them
        if W is None or H is None:
            (H, W) = frame.shape[:2]
            
        # clone the output frame, then convert it from BGR to RGB
        # ordering, resize the frame to a fixed 224x224, and then
        # perform mean subtraction
        output = frame.copy()
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = cv2.resize(frame, (224, 224)).astype("float32")
        frame -= mean
        
        # make predictions on the frame and then update the predictions
        # queue
        preds = model.predict(np.expand_dims(frame, axis=0))[0]
        Q.append(preds)
        
        # perform prediction averaging over the current history of
        # previous predictions
        results = np.array(Q).mean(axis=0)
        i = np.argmax(results)
        label = lb.classes_[i]
        
        # draw the activity on the output frame
        text = "activity: {}".format(label)
        cv2.putText(output, text, (35, 50), cv2.FONT_HERSHEY_SIMPLEX,
            1.25, (0, 255, 0), 5)
        
        # check if the video writer is None
        if writer is None:
            
            # initialize our video writer
            fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
            writer = cv2.VideoWriter("out_vid_3.mp4", fourcc, 30,
                (W, H), True)
            
        # write the output frame to disk
        writer.write(output)

# release the file pointers1
print("[INFO] cleaning up...")
writer.release()
vs.release()

1/1 [==============================] - 0s 22ms/step


1/1 [==============================] - 0s 23ms/step


1/1 [==============================] - 0s 22ms/step


1/1 [==============================] - 0s 22ms/step


1/1 [==============================] - 0s 25ms/step


1/1 [==============================] - 0s 21ms/step


1/1 [==============================] - 0s 21ms/step
[INFO] cleaning up...
